# 实时预览ass字幕

# 字幕文本超过屏幕宽度

## 参考链接

* [Poe-ass字幕如何设置最大宽度，超过自动换行](https://poe.com/s/wldjpFM4L9p5R66gX95k)

# 字幕样式
这里做以下几件事:

1.  使用文本`outputs/我的新朋友/subtitle.ass`作为演示字幕
2. 使用`pysubs2`修改字幕文件

In [ ]:
import gradio as gr

def update_style(font_type, font_size, bold, italic, underline, strikethrough, primary_color, secondary_color, border_color, shadow_color):
    style = {
        "font-family": font_type,
        "font-size": f"{font_size}px",
        "font-weight": "bold" if bold else "normal",
        "font-style": "italic" if italic else "normal",
        "text-decoration": f"{'underline ' if underline else ''}{'line-through ' if strikethrough else ''}",
        "color": primary_color,
        "background-color": secondary_color,
        "border-color": border_color,
        "text-shadow": f"2px 2px {shadow_color}"
    }
    return style

def create_gradio_interface():
    with gr.Blocks() as demo:
        with gr.Accordion("字体设置"):
            font_type = gr.Dropdown(label="字体类型", choices=["Arial", "Times New Roman", "Courier New"])
            font_size = gr.Slider(label="字体大小", minimum=10, maximum=50, value=16)
            bold = gr.Checkbox(label="加粗")
            italic = gr.Checkbox(label="斜体")
            underline = gr.Checkbox(label="下划线")
            strikethrough = gr.Checkbox(label="删除线")
        
        with gr.Accordion("颜色设置"):
            primary_color = gr.ColorPicker(label="主要颜色")
            secondary_color = gr.ColorPicker(label="次要颜色")
            border_color = gr.ColorPicker(label="边框颜色")
            shadow_color = gr.ColorPicker(label="字体阴影颜色")
        
        style_output = gr.JSON(label="样式输出")
        
        inputs = [font_type, font_size, bold, italic, underline, strikethrough, primary_color, secondary_color, border_color, shadow_color]
        style_output.change(update_style, inputs=inputs, outputs=style_output)
    
    demo.launch()

create_gradio_interface()

# 位置自适应
已知如下条件:
1. 屏幕分辨率1080x1920
2. 字幕文本为`One sunny day, Lily found a tiny kitten in her garden\N一个阳光明媚的日子，莉莉在花园里发现了一只小猫`
3. 字体名称为Arial
4. 字体大小为28

思路如下:

1. 首先要有一个方法能够根据字体、大小和内容来获取文本的宽度
2. 拿到宽度后计算它在1080宽度的屏幕中居中时的x位置
3. 最后生成 f"{{\\pos({pos_x},100)}}" + 字幕文本

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

def get_text_width(text, font_name, font_size):
    font = FontProperties(family=font_name, size=font_size)
    fig, ax = plt.subplots()
    text_obj = ax.text(0, 0, text, fontproperties=font)
    renderer = fig.canvas.get_renderer()
    bbox = text_obj.get_window_extent(renderer=renderer)
    width = bbox.width
    plt.close(fig)
    return width

def calculate_centered_position(text_width, screen_width=1080):
    return (screen_width - text_width) / 2

text = "One sunny day, Lily found a tiny kitten in her garden\\N一个阳光明媚的日子，莉莉在花园里发现了一只小猫"
font_name = r"D:\Workspace\aiworld\自动化工作流\英语短文\py_automation\jianying\resources\fonts\华文中宋.ttf"
font_size = 28

text_width = get_text_width(text, font_name, font_size)
pos_x = calculate_centered_position(text_width)
subtitle_text = f"{{\\pos({pos_x},100)}}" + text

print(subtitle_text)